In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../../")

In [ ]:
from fishjaw.util import util

config = util.userconf()

In [ ]:
"""Read in a small number of CT scans and the corresponding labels"""

import pathlib


dicom_dir = pathlib.Path("../../dicoms/Training set 2/")
dicom_paths = sorted(list(dicom_dir.glob("*.dcm")))
print(len(dicom_paths))

In [ ]:
train_paths, val_paths, test_paths = (
    dicom_paths[:15],
    dicom_paths[15:18],
    dicom_paths[18:],
)

In [ ]:
from tqdm import tqdm
from fishjaw.images import io

train_imgs, train_labels = zip(*[io.read_dicom(path) for path in tqdm(train_paths)])
test_imgs, test_labels = zip(*[io.read_dicom(path) for path in tqdm(test_paths)])
val_imgs, val_labels = zip(*[io.read_dicom(path) for path in tqdm(val_paths)])

In [ ]:
from scipy.ndimage import center_of_mass

train_centroids = [
    tuple(map(int, center_of_mass(label))) for label in tqdm(train_labels)
]
val_centroids = [tuple(map(int, center_of_mass(label))) for label in tqdm(val_labels)]
test_centroids = [tuple(map(int, center_of_mass(label))) for label in tqdm(test_labels)]

In [ ]:
"""Plot the centroids"""

import matplotlib.pyplot as plt

fig, axis = plt.subplots(figsize=(6, 6))

img_slice = train_imgs[0][train_centroids[0][0]]
label_slice = train_labels[0][train_centroids[0][0]]

axis.imshow(img_slice, cmap="gray")
axis.imshow(label_slice, cmap="afmhot_r", alpha=0.3)

axis.scatter(
    train_centroids[0][2], train_centroids[0][1], color="red", s=20, label="Centroid"
)
axis.axis("off")

In [ ]:
# Since we don't really need all the information of the high-resolution images,
# we can downsample them to speed up processing.
import numpy as np

# ndimage.zoom is faster but i think skimage.resize is more accurate
from scipy.ndimage import zoom

output_size = (512, 128, 128)


def resize_image_and_label(imgs, labels, centroids, target_shape):
    scale_factors = []
    resized_imgs = []
    resized_labels = []
    scaled_centroids = []

    for img, label, centroid in tqdm(
        zip(imgs, labels, centroids, strict=True), total=len(imgs)
    ):
        assert img.shape == label.shape, "Image and label must have the same shape"

        # Calculate scale factors for each dimension
        scale_factor = tuple(
            target_dim / orig_dim
            for orig_dim, target_dim in zip(img.shape, output_size)
        )
        scale_factors.append(scale_factor)

        # Resize image and label using zoom
        resized_img = zoom(
            img, scale_factor, order=3
        )  # Use cubic interpolation for images
        resized_label = zoom(
            label, scale_factor, order=0
        )  # Use nearest-neighbor for labels

        resized_imgs.append(resized_img)
        resized_labels.append(resized_label)

        # Rescale centroid to match the resized image
        scaled_centroid = tuple(int(c * sf) for c, sf in zip(centroid, scale_factor))
        scaled_centroids.append(scaled_centroid)

    print(np.mean(scale_factors, axis=0), np.std(scale_factors, axis=0))

    return resized_imgs, resized_labels, scaled_centroids, scale_factors


train_imgs, train_labels, train_centroids, _ = resize_image_and_label(
    train_imgs, train_labels, train_centroids, output_size
)
val_imgs, val_labels, val_centroids, _ = resize_image_and_label(
    val_imgs, val_labels, val_centroids, output_size
)

# Keep the test set at original resolution for evaluation
resized_test_imgs, resized_test_labels, resized_test_centroids, test_scale_factors = (
    resize_image_and_label(test_imgs, test_labels, test_centroids, output_size)
)

In [ ]:
fig, axis = plt.subplots(figsize=(6, 6))

img_slice = resized_test_imgs[0][resized_test_centroids[0][0]]
label_slice = resized_test_labels[0][resized_test_centroids[0][0]]

axis.imshow(img_slice, cmap="gray")
axis.imshow(label_slice, cmap="afmhot_r", alpha=0.3)

axis.scatter(
    resized_test_centroids[0][2],
    resized_test_centroids[0][1],
    color="red",
    s=20,
    label="Centroid",
)
axis.axis("off")

In [ ]:
# Define a model arch

from monai.networks.nets import AttentionUnet

model = AttentionUnet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    strides=(2, 2, 2),
    channels=(4, 8, 16, 32),
    dropout=0.05,
).to("cuda")

In [ ]:
# Training loop
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

from scipy.ndimage import gaussian_filter


class ImgDataset(Dataset):
    def __init__(self, images, centroids, sigma):
        self.data = torch.tensor(
            np.array(images, dtype=np.float32), dtype=torch.float32
        ).unsqueeze(1)

        self.centroids = []
        for img, centroid in zip(images, centroids):
            gaussian_mask = np.zeros_like(img, dtype=np.float32)
            gaussian_mask[centroid[0], centroid[1], centroid[2]] = 1  # Set the centroid
            gaussian_mask = gaussian_filter(
                gaussian_mask, sigma=sigma
            )  # Apply Gaussian filter

            # Normalise to sum to 1 to make it a valid probability distribution
            gaussian_mask /= np.sum(gaussian_mask) if np.sum(gaussian_mask) > 0 else 1

            self.centroids.append(gaussian_mask)

        self.centroids = torch.tensor(
            np.array(self.centroids), dtype=torch.float32
        ).unsqueeze(1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx].to("cuda"), self.centroids[idx].to("cuda")


# Initialize
sigma = 2
optimizer = optim.Adam(model.parameters(), lr=1e-5)

train_dataset = ImgDataset(train_imgs, train_centroids, sigma=sigma)
val_dataset = ImgDataset(val_imgs, val_centroids, sigma=sigma)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Plot the first thing in the dataloader


def plot_sample(img, centroid):
    # Find the index of the Z slice where the centroid has the highest sum
    centre = torch.argmax(centroid[0][0].sum(dim=(1, 2))).item()

    fig, axis = plt.subplots(figsize=(6, 6))

    img_slice = img[0][0][centre].cpu().numpy()
    centroid_slice = centroid[0][0][centre].cpu().numpy()
    axis.imshow(img_slice, cmap="gray")
    axis.imshow(centroid_slice, cmap="afmhot_r", alpha=0.3)

    axis.axis("off")


plot_sample(*next(iter(train_loader)))

In [ ]:
# Training loop
import torch.nn.functional as F


def kl_heatmap_loss(pred, target):
    # Apply log-softmax to predictions
    pred = F.log_softmax(pred.view(pred.size(0), -1), dim=1)

    # Ensure target is normalized (if not already)
    target = target.view(pred.size(0), -1)
    target = target / target.sum(dim=1, keepdim=True)

    # Compute KL divergence
    return F.kl_div(pred, target, reduction="batchmean")


def train(model, train_data, val_data, n_epochs):
    model.train()
    train_losses, val_losses = [], []

    pbar = tqdm(range(n_epochs), total=n_epochs, desc="Training Epochs")
    for epoch in pbar:
        train_loss, val_loss = [], []

        for i, (volumes, coords) in enumerate(train_data):
            optimizer.zero_grad()
            outputs = model(volumes)
            loss = kl_heatmap_loss(outputs, coords)
            loss.backward()

            optimizer.step()

            train_loss.append(loss.item())

        for i, (volumes, coords) in enumerate(val_data):
            with torch.no_grad():
                outputs = model(volumes)
                loss = kl_heatmap_loss(outputs, coords)
                val_loss.append(loss.item())

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        pbar.set_postfix(train_loss=np.mean(train_loss), val_loss=np.mean(val_loss))

    return train_losses, val_losses


n_epochs = 100
train_losses, val_losses = train(model, train_loader, val_loader, n_epochs)

In [ ]:
from fishjaw.visualisation import training

fig = training.plot_losses(train_losses, val_losses)

In [ ]:
# Test the model - visualise the heatmap
model.eval()
with torch.no_grad():
    test_vol = (
        torch.tensor(
            np.array(resized_test_imgs[0], dtype=np.float32), dtype=torch.float32
        )
        .unsqueeze(0)
        .unsqueeze(0)
        .to("cuda")
    )

    predicted_heatmap = model(test_vol)

    plot_sample(test_vol, predicted_heatmap)

    # Repeat but rotate to have different axis orientations
    plot_sample(
        test_vol.permute(0, 1, 3, 2, 4), predicted_heatmap.permute(0, 1, 3, 2, 4)
    )
    plot_sample(
        test_vol.permute(0, 1, 4, 2, 3), predicted_heatmap.permute(0, 1, 4, 2, 3)
    )

In [ ]:
# Get the predicted co-ords from the maximum along each axis
def find_gaussian_center(heatmap):
    """
    Find the center of a 3D Gaussian heatmap using a convolution-based approach.

    Args:
        heatmap (torch.Tensor): A 5D tensor of shape (batch, channel, depth, height, width).

    Returns:
        List[Tuple[int, int, int]]: The indices of the center (z, y, x) for each heatmap in the batch.
    """
    # Define a 3D Gaussian kernel (or a box kernel)
    kernel_size = 5  # Size of the kernel
    sigma = 1.0  # Standard deviation for the Gaussian kernel
    coords = torch.arange(kernel_size, device=heatmap.device) - (kernel_size - 1) / 2
    z, y, x = torch.meshgrid(coords, coords, coords, indexing="ij")
    gaussian_kernel = torch.exp(-(z**2 + y**2 + x**2) / (2 * sigma**2))
    gaussian_kernel = gaussian_kernel / gaussian_kernel.sum()  # Normalize the kernel
    gaussian_kernel = gaussian_kernel.view(
        1, 1, *gaussian_kernel.shape
    )  # Add batch and channel dims

    # Apply the convolution
    smoothed_heatmap = F.conv3d(heatmap, gaussian_kernel, padding=kernel_size // 2)

    # Find the voxel with the maximum value
    flat_idx = torch.argmax(smoothed_heatmap.view(smoothed_heatmap.size(0), -1), dim=1)
    batch_size, _, depth, height, width = heatmap.shape
    z = flat_idx // (height * width)
    y = (flat_idx % (height * width)) // width
    x = flat_idx % width

    # Combine the coordinates into a list of tuples
    centers = [(z[i].item(), y[i].item(), x[i].item()) for i in range(batch_size)]
    return centers


(predicted_coords,) = find_gaussian_center(predicted_heatmap)
predicted_coords

In [ ]:
# Plot slices of the cropped jaw
fig, axis = plt.subplots(1, 1)

img_slice = resized_test_imgs[0][int(predicted_coords[0])]
label_slice = resized_test_labels[0][int(predicted_coords[0])]

axis.imshow(img_slice, cmap="gray")
axis.imshow(label_slice, cmap="afmhot_r", alpha=0.3)
axis.scatter(
    predicted_coords[2],
    predicted_coords[1],
    color="red",
    s=20,
    label="Predicted Centroid",
)
axis.axis("off")

In [ ]:
# Scale the prediction back up
scaled_prediction = tuple(
    int(coord / scale_factor)
    for coord, scale_factor in zip(predicted_coords, test_scale_factors[0])
)
print(f"Scaled prediction: {scaled_prediction}")

fig, axis = plt.subplots(1, 1)

img_slice = test_imgs[0][int(scaled_prediction[0])]
label_slice = test_labels[0][int(scaled_prediction[0])]

axis.imshow(img_slice, cmap="gray")
axis.imshow(label_slice, cmap="afmhot_r", alpha=0.3)
axis.scatter(
    scaled_prediction[2],
    scaled_prediction[1],
    color="red",
    s=20,
    label="Predicted Centroid",
)
axis.axis("off")